In [20]:
import time
from ore_algebra import *
DOP, t, D = DifferentialOperators()
# The following may be overridden in input file, but need not be specified there
#
#
defaultPrecision=1e-40;
# Do not forget to increase the precision of initial conditions if going beyond e-100
#
#
#
precision=[]; #if empty we will use the default precision for all systems
periodsOnly=true; 
paths=[]; #unless overridden we will convert this to a list of straight paths

#load("~/git-projects/periods/integrate/data/current.sage")
load("~/git-projects/period-suite/current.sage")
steps=len(allODEs);
cohomologies=[];
allTransMats=[1..steps];

if len(paths)==0:
    paths=[[0,1] for j in [1..steps]]
    
if len(precision)==0:
    precision=[defaultPrecision for j in [1..steps]]


t0=time.time();


print "Integrating systems", 1, "through", steps;
for j in [1..steps]:
    if periodsOnly and j==steps:
        noOfdeqs=geometricGenus;
    else:
        noOfdeqs=len(allODEs[j-1]);
    deqs=[deq.numerator() for deq in allODEs[j-1][0:noOfdeqs]];
    print "";
    print "Integrating system number", j;
    print "";
    %time transitionMatricies = [deqs[i-1].numerical_transition_matrix(paths[j-1], precision[j-1],assume_analytic=true) for i in [1..noOfdeqs]]
    allTransMats[j-1]=transitionMatricies;
    print "Maximal error in transition matricies:"
    print [max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies]
    print "The largest error:"
    print max([max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies])
    cohomology=[1..noOfdeqs];
    for i in [1..noOfdeqs]:
        if j==1:
            init=Matrix(inits[i-1]);
            a = init.nrows(); b = init.ncols();
            init =  MatrixSpace(ComplexBallField(2000), a, b)(init);
        else:
            init=Matrix(rawInits[j-2][i-1])*cohomologies[j-2];
        transitionMatrix=transitionMatricies[i-1];
        cohomology[i-1]=(transitionMatrix * init).row(0);
    cohomologies=cohomologies+[Matrix(cohomology)];
    
periodsWithError=Matrix(cohomologies[-1].rows()[0:geometricGenus]);
maximalError=max(periodsWithError.apply_map(lambda x : x.diameter()).list());
periods=periodsWithError.apply_map(lambda x : x.mid());
print "\nAccumulated maximal error:", maximalError
t1=time.time();
print "\nIt took", t1-t0, "seconds in total.\n"
print periods


Integrating systems 1 through 1

Integrating system number 1



KeyboardInterrupt: 

In [11]:
#### Only for K3s
##
## Try a few values for "scale", do not exceed the negative exponent of accumulated error
##
scale = 20
##
## verify that you found honest relations by checking suspected relations using more digits
##
verify= 120
##
##

load("K3_intersection_matrix.sage")

periodVector=periods*intersectionMatrix.inverse()
u1 = periodVector.apply_map(real)[0].list()
u2 = periodVector.apply_map(imag)[0].list()
M = matrix([u1,u2])

proj = M.transpose()
Proj = (10^scale*proj).apply_map(lambda x : 10^3*x.round())
lattice = block_matrix([[Proj, matrix.identity(Proj.dimensions()[0])]])
reducedLattice = lattice.LLL()

Proj2 = (10^verify*proj).apply_map(lambda x : x.round())
testRelations = [v[2:23] * Proj2 for v in reducedLattice.rows()]

norms1 = [reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]
norms2 = [testRelations[i].norm().n()+reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]

norms=norms1

sortedNorms=sorted(norms)
consecutiveRatios = [sortedNorms[i]/sortedNorms[i-1] for i in [1..20]]

j=0; rank=1;
while j <= 19 :
    if consecutiveRatios[j] > 100 :
        j=20
    rank = rank + 1
    j=j+1
    
if sortedNorms[0] > 50 :
    rank = 1

print "\nPicard rank appears to be", rank, "\n"
print "Check to see if this makes sense:\n"

print "01","||", "---hyperplane section---"
j=0
while j <= 7 :
    print "0" + str(j+2), "||", norms[j], testRelations[j]
    j=j+1
while j <= 20 :
    print j+2, "||", norms[j], testRelations[j]
    j=j+1



Picard rank appears to be 15 

Check to see if this makes sense:

01 || ---hyperplane section---
02 || 21.0237960416286 (16703008872482861189850114587503230707834195288018651362691756493141468950560768, 4631683569492647816942839400347516314130799386625622561578303360316525185597440)
03 || 34.8425027803687 (133392486801388257127953774730008469846967022334817929773455136777115925345206272, 255668933035994159495244734899182900540020126141734365399122345489472190244978688)
04 || 36.6333181680284 (37053468555941182535542715202780130513046395093004980492626426882532201484779520, 88001987820360308521913948606602809968485188345886828669987763846013978526351360)
05 || 43.5660418215839 (3357970587882169667283558565251949327744829555303576357144269936229480759558144, -24084754561361768648102764881807084833480156810453237320207177473645930965106688)
06 || 377.695644666443 (-668844055457047673815401902156433527737450748924906308032916869625708215082680320, -3325548802895721132564958689449516713545

In [10]:
periods.nrows()

1